In [1]:
import datetime
import json
import numpy as np
import psycopg2
import requests
import time
import uuid

from typing import Any

In [2]:
PROJECT_ID = 10
NUM_LOGS = 50000
NUM_SEARCH_QUERIES = 20
NUM_GET_LOGS_QUERIES = 20
POSTGRES_CONFIG = {
    "user": "rpuser",
    "password": "rppass",
    "host": "localhost",
    "port": 5432,
    "database": "reportportal",
}

In [3]:
def generate_logs(num):
    all_logs = []
    log_messages = ["this is a test log", "this is a different log"]
    for i in range(num):
        all_logs.append(
            {
                "uuid": str(uuid.uuid4()),
                "log_time": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "log_message": log_messages[np.random.randint(0, len(log_messages))],
                "item_id": int(np.random.randint(1000, 1010)),
                "launch_id": int(np.random.randint(100, 501)),
                "last_modified": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "log_level": int(np.random.choice([20000, 30000, 40000, 50000])),
                "attachment_id": int(np.random.randint(1000, 5001)),
            }
        )
    return all_logs

In [4]:
def make_logs_post_request(endpoint: str, body: Any):
    return requests.post(
        f"http://localhost:5010/{endpoint}",
        data=json.dumps(body),
        headers={"Content-type": "application/json", "Accept": "text/plain"},
    ).__dict__

In [5]:
def add_random_logs(num_logs: int, project_id: int, drop_existing: bool = True):
    if drop_existing:
        print("Dropping existing logs: ", end="")
        print(make_logs_post_request("delete_project", project_id)["reason"])
    logs = generate_logs(num_logs)
    print("Indexing random logs: ", end="")
    print(make_logs_post_request("index_logs", {"logs": logs, "project": project_id})["reason"])

In [6]:
def make_search_queries(num_queries: int, project_id: int):
    queries = ["log", "test", "different"]
    print("Making search queries: ", end="")
    for _ in range(num_queries):
        search_response = make_logs_post_request(
            "search_logs",
            {
                "query": queries[np.random.randint(0, len(queries))],
                "project": project_id,
            },
        )
        if search_response["status_code"] != 200:
            print(search_response["reason"])
            return
    print("OK")

In [7]:
def make_get_logs_by_test_item_queries(num_queries: int, project_id: int):
    print("Making get logs by test item queries: ", end="")
    for _ in range(num_queries):
        get_logs_response = make_logs_post_request(
            "get_logs_by_test_item",
            {
                "test_item": np.random.randint(1000, 1010),
                "project": project_id,
            },
        )
        if get_logs_response["status_code"] != 200:
            print(get_logs_response["reason"])
            return
    print("OK")

In [8]:
def drop_search_indices():
    query = """
        DROP INDEX rp_log_ti_idx;
        DROP INDEX rp_log_message_trgm_idx;
    """
    connection = psycopg2.connect(**POSTGRES_CONFIG)
    cursor = connection.cursor()
    print("Dropping search indices: ", end="")
    try:
        cursor.execute(query)
        try:
            print(cursor.fetchall())
        except psycopg2.ProgrammingError:
            print("OK")
    except psycopg2.ProgrammingError as err:
        print(err)
    connection.commit()
    cursor.close()
    connection.close()

In [9]:
def test_querying_performance(
    num_logs: int,
    num_search_queries: int,
    num_get_logs_queries: int,
    drop_indices: bool,
    project_id: int,
):
    print(f"Testing querying performance for drop_indices = {drop_indices}")
    add_random_logs(num_logs=num_logs, project_id=project_id, drop_existing=True)
    if drop_indices:
        drop_search_indices()

    print()
    start = time.time()
    make_search_queries(num_queries=num_search_queries, project_id=project_id)
    print(f"Time for search: {time.time() - start} s")

    print()
    start = time.time()
    make_get_logs_by_test_item_queries(
        num_queries=num_get_logs_queries, project_id=project_id
    )
    print(f"Time for get logs by test item: {time.time() - start} s")

In [10]:
test_querying_performance(
    num_logs=NUM_LOGS,
    num_search_queries=NUM_SEARCH_QUERIES,
    num_get_logs_queries=NUM_GET_LOGS_QUERIES,
    drop_indices=False,
    project_id=PROJECT_ID
)

Testing querying performance for drop_indices = False
Dropping existing logs: OK
Indexing random logs: OK

Making search queries: OK
Time for search: 225.31177139282227 s

Making get logs by test item queries: OK
Time for get logs by test item: 47.098653078079224 s


In [11]:
test_querying_performance(
    num_logs=NUM_LOGS,
    num_search_queries=NUM_SEARCH_QUERIES,
    num_get_logs_queries=NUM_GET_LOGS_QUERIES,
    drop_indices=True,
    project_id=PROJECT_ID
)

Testing querying performance for drop_indices = True
Dropping existing logs: OK
Indexing random logs: OK
Dropping search indices: OK

Making search queries: OK
Time for search: 206.5304319858551 s

Making get logs by test item queries: OK
Time for get logs by test item: 47.27680420875549 s


In [12]:
make_logs_post_request("delete_project", PROJECT_ID)

{'_content': b'1\n',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Content-Type': 'application/json', 'Content-Length': '2', 'Access-Control-Allow-Origin': '*', 'Server': 'Werkzeug/1.0.1 Python/3.7.4', 'Date': 'Fri, 07 May 2021 12:04:16 GMT'},
 'raw': <urllib3.response.HTTPResponse at 0x182c64f2b08>,
 'url': 'http://localhost:5010/delete_project',
 'encoding': None,
 'history': [],
 'reason': 'OK',
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(seconds=2, microseconds=114601),
 'request': <PreparedRequest [POST]>,
 'connection': <requests.adapters.HTTPAdapter at 0x182c6503d88>}